# Imports

In [3]:
import numpy as np
import pandas as pd

from functools import reduce


import plotly.express as px


import warnings
warnings.filterwarnings('ignore')

In [4]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


In [6]:
dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung']
    
    
    

## ToDo

- % Zuwachs: Either since 2015 or Benutzerdefiniert
- Waterfall graphs for %, stacked barchart for Gesamtvolum
- Top/Bottom 10
- DF to the bottom

Filter Steps:
 1. Only BMZ
 2. 100% of full, 50% of Partial Financing

# World

In [54]:
dictionary_df['2022'][['Amount']].sum()*1_000_000

Amount    1.206876e+10
dtype: float64

### Basic Processing of Data


In [97]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



In [104]:
df_merged

,amount_2013,clim_rel_amount_2013,clim_rel_percent_2013,amount_2014,clim_rel_amount_2014,clim_rel_percent_2014,amount_2015,clim_rel_amount_2015,clim_rel_percent_2015,amount_2016,...,clim_rel_percent_2019,amount_2020,clim_rel_amount_2020,clim_rel_percent_2020,amount_2021,clim_rel_amount_2021,clim_rel_percent_2021,amount_2022,clim_rel_amount_2022,clim_rel_percent_2022
Recipient Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,229.878097,18.110155,0.078782,286.807860,30.496058,0.106329,197.429500,30.311894,0.153533,346.234112,...,0.165281,196.114075,33.459576,0.170613,352.840862,47.657621,0.135068,201.939382,12.035070,0.059597
"Africa, regional",132.706037,34.237272,0.257993,129.848523,15.707783,0.120970,182.614727,75.974172,0.416035,129.000894,...,0.234096,686.808006,161.181838,0.234683,643.229422,131.993625,0.205205,627.644704,138.891343,0.221290
Albania,19.317268,5.281618,0.273414,16.371641,4.651060,0.284092,19.155382,6.362595,0.332157,18.968980,...,0.271722,30.080577,5.804841,0.192976,29.366152,7.881105,0.268374,25.328362,8.592429,0.339241
Algeria,6.080655,1.175290,0.193283,5.218615,2.543035,0.487301,6.014444,3.463474,0.575859,4.684949,...,0.455295,11.714897,4.716805,0.402633,25.777176,6.526832,0.253202,16.273651,7.171532,0.440684
"America, regional",61.039902,12.957501,0.212279,73.783053,20.492112,0.277735,48.050705,5.792946,0.120559,77.236341,...,0.391508,146.314609,54.032500,0.369290,82.869791,29.554400,0.356637,107.665968,50.903984,0.472795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Central America, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.723529,6.512341,3.309877,0.508247,7.760249,3.882104,0.500255,9.319793,4.376188,0.469558
"Eastern Africa, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.165010,27.979557,6.321787,0.225943,31.071347,2.476874,0.079716,31.393523,3.239855,0.103201
"Middle Africa, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.470844,5.156201,1.411457,0.273740,7.093164,1.752717,0.247099,7.352916,1.580488,0.214947


In [103]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


globe_df = globe_df.loc[[142]]

globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
142,5627.252576,845.563368,6872.547503,1036.792467,5880.075415,995.181229,7376.387963,1352.222299,7670.370692,1457.414366,8497.391449,1659.672747,9125.561581,2007.334822,10818.981966,2195.044259,11116.566037,2490.046211,12068.758817,2830.26513


### Prepare data for stacked barchart


In [105]:

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000



melted_df.to_csv("../upload_data/globe_df")

### Plot stacked bar

In [106]:
read_df = pd.read_csv('../upload_data/globe_df')

fig = px.bar(read_df, x='Year', y='Amount', color='Type',
            title='Global Financing Totals',
            labels={'Amount': 'Financing Amount (Euros)', 'Year': 'Year'},
            category_orders={'Type': ['Other Funds','Climate Finance']},
            color_discrete_map={'Other Funds': 'orange', 'Climate Finance': 'green'})# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

### Percentage

In [108]:
years = [col.split('_')[-1] for col in globe_df.columns if col.startswith('amount_')]

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    clim_rel_percent_col = f'clim_rel_percent_{year}'
    
    # Calculate the clim_rel_percent column for the year
    # Note: It's a good practice to handle division by zero or check if 'amount_' is not zero to avoid ZeroDivisionError.
    globe_df[clim_rel_percent_col] = (globe_df[clim_rel_amount_col] / globe_df[amount_col]) * 100
    
    

In [110]:
selected_columns = [col for col in globe_df.columns if col.startswith('clim_rel_percent_')]
filtered_df = globe_df[selected_columns]

In [122]:

# Melt the DataFrame to get it in the right shape for plotting
melted_df = filtered_df.reset_index().melt(id_vars=['index'], var_name='Year', value_name='Percentage')

# Clean the 'Year' column to keep only the year part
melted_df['Year'] = melted_df['Year'].str.replace('clim_rel_percent_', '')

melted_df = melted_df.drop(columns = 'index')

In [123]:
melted_df

,Year,Percentage
0,2013,15.026220
1,2014,15.085999
2,2015,16.924634
3,2016,18.331768
4,2017,19.000573
5,2018,19.531556
6,2019,21.996836
7,2020,20.288824
8,2021,22.399419
9,2022,23.451170


In [124]:
import plotly.graph_objects as go


In [125]:
melted_df['Change'] = melted_df['Percentage'].diff()

In [131]:
melted_df.to_csv('../upload_data/globe_waterfall.csv')

In [134]:
globe_waterfall_df = pd.read_csv('../upload_data/globe_waterfall.csv')

initial_value = globe_waterfall_df['Percentage'].iloc[0] - globe_waterfall_df['Change'].iloc[1] # Subtract the first actual change to get the starting point


fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = ["absolute"] + ["relative"] * (len(df) - 1), # The first measure is absolute, others are relative
    x = globe_waterfall_df['Year'].astype(str),
    textposition = "outside",
    text = globe_waterfall_df['Change'].round(2).astype(str),
    y = [initial_value] + globe_waterfall_df['Change'].tolist()[1:], # The initial value plus the changes
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Yearly Percentage Change",
        xaxis = {"type":"category"},
        yaxis = {"title":"Percentage"},
)

fig.show()

# Country Sum

# Country Compare

# Country Breakdown

In [ ]:
use_df = df_dictionary['2022'][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [ ]:

use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

In [ ]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

In [ ]:
use_df['clim_rel'] = use_df.apply(determine_value, axis=1)


In [ ]:
use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']

In [ ]:
grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]

In [ ]:
grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']

In [ ]:
grouped_df

In [ ]:
processed_grouped_dfs = []

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']].reset_index()
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_processed_df = grouped_df.rename(columns={'clim_rel_percent': key})

    processed_grouped_dfs.append(grouped_processed_df[['Recipient Name',key]])

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


# df_long.to_csv('../upload_data/country_specific_df.csv')



In [ ]:
df_long.to_csv('../upload_data/newmethod_country_specific_df.csv')


In [ ]:
df_long

In [ ]:
processed_grouped_dfs = []

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']

    use_df.loc[use_df['Climate Relevance'] > 1, 'Climate Relevance'] = 1

    first_group_df = use_df.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']].reset_index()

    df_grouped = first_group_df.groupby('Recipient Name').apply(
        lambda x: (x[x['Climate Relevance'] == 1]['Amount'].sum()\
                   / x['Amount'].sum()) * 100).reset_index(name='Climate Relevance %')

    grouped_processed_df = df_grouped.rename(columns={'Climate Relevance %': key})

    processed_grouped_dfs.append(grouped_processed_df)

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


df_long.to_csv('../upload_data/country_specific_df.csv')



In [ ]:
df_long

In [ ]:
df_long = pd.read_csv('../upload_data/country_specific_df.csv')[['Recipient Name', 'Year', 'Value']]

In [ ]:
df_long

In [ ]:
countries = df_long['Recipient Name'].unique()
# countries

In [ ]:
selected_df = df_long[df_long['Recipient Name'].isin(['India', 'Brazil', 'Ukraine', 'Namibia', 'South Africa'])]

In [ ]:
fig = px.line(selected_df, 
              x='Year', 
              y='Value', 
              color='Recipient Name',
              title='Value by Year for Selected Country',
              labels={'Value': 'Climate Relevant Financing (%)', 'Year': 'Year'},
              markers=True)

fig.show()

# Globe Graph

In [ ]:
def df_compiler(dictionary_of_df):
    
    processed_grouped_dfs = []
    
    for key in dictionary_of_df.keys():
        
        use_df = dictionary_of_df[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
        
        use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
        use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
        
        use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']
        
        use_df.loc[use_df['Climate Relevance'] > 2, 'Climate Relevance'] = 2
        
        use_df['Amount'] = use_df['Amount'] * 1_000_000
        
        grouped_processed_df = use_df.groupby(['Climate Relevance']).sum()[['Amount']]
        
        grouped_processed_df = grouped_processed_df.rename(columns={'Amount': key})
        
        processed_grouped_dfs.append(grouped_processed_df)
        
    merged_df = pd.concat(processed_grouped_dfs, axis=1)
    
    sum_row = merged_df.sum()
    sum_df = pd.DataFrame([sum_row], index=['Total Financing'])
    
    merged_df = merged_df.append(sum_df)

    merged_df.index = ['No Climate Relevance','Partial Climate Relevance',
                       'High Climate Relevance', 'Total Financing']

    return merged_df


merged_df = df_compiler(df_dictionary)

merged_df.to_csv('../upload_data/global_df.csv')

merged_df

In [ ]:
globe_df = pd.read_csv('../upload_data/global_df.csv')
globe_df

In [ ]:

df_long = merged_df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']



fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Global BMZ Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:
df_long = df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']

# Filter out the 'Total Financing' row to avoid double counting in the stacked bar
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']

# Create the stacked bar chart
fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Total Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:


testing_2022 = df_2022[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [ ]:
testing_2022['Climate Adaptation'] = testing_2022['Climate Adaptation'].fillna(0)
testing_2022['Climate Mitigation'] = testing_2022['Climate Mitigation'].fillna(0)

In [ ]:
testing_2022['Climate Relevance'] = testing_2022['Climate Adaptation'] + testing_2022['Climate Mitigation']

In [ ]:
testing_2022.loc[testing_2022['Climate Relevance'] > 2, 'Climate Relevance'] = 2


In [ ]:
testing_2022.groupby(['Climate Relevance']).sum()[['Amount']].rename(columns={'Amount': '2022'})

In [ ]:
grouped_test = testing_2022.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']]